In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.datasets import fashion_mnist, mnist
from tensorflow.keras.backend import clear_session
from sklearn.utils import resample
import numpy as np
import pandas as pd
import seaborn as sns

In [5]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [18]:
(X_train_org, y_train_org), (X_test_org, y_test_org) = fashion_mnist.load_data()

X_train_org = X_train_org.reshape(-1, 28, 28, 1)
X_test_org = X_test_org.reshape(-1, 28, 28, 1)

X_train_org = X_train_org.astype('float32')
X_test_org = X_test_org.astype('float32')
X_train_org /= 255
X_test_org /= 255

In [ ]:
# 1,2,5,10 conv2d
# 5, 10, 15 epok
# 100, 1000 wag
# 2500, 5000, 10000 sampli, 0.8 ratio


In [30]:
def gen_3a():
    for ratio in np.arange(0.05, 1.0, 0.05):
        clear_session()

        train_samples = int(10000 * ratio)
        test_samples = 10000 - train_samples
        X_train, y_train = resample(X_train_org, y_train_org, random_state=0, n_samples=train_samples)
        X_test, y_test = resample(X_test_org, y_test_org, random_state=0, n_samples=test_samples)

        model = Sequential([
            Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
            MaxPooling2D((2, 2), strides=2),
            Conv2D(64, (3,3), padding='same', activation='relu'),
            MaxPooling2D((2, 2), strides=2),
            Flatten(),
            Dense(128, activation='relu'),
            Dense(10, activation='softmax')
        ])

        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
        loss, accuracy = model.evaluate(X_test, y_test, verbose=False)

        print("===============================================")
        print(train_samples, "-", test_samples)
        print({'accuracy': accuracy, 'loss': loss})
        print()
        yield ratio, accuracy, loss

results_3a = pd.DataFrame(gen_3a(), columns=['ratio', 'accuracy', 'loss'])

Train on 500 samples, validate on 9500 samples
Epoch 1/10
500/500 [==============================] - 1s 2ms/sample - loss: 1.6289 - accuracy: 0.4460 - val_loss: 1.1330 - val_accuracy: 0.5783
Epoch 2/10
500/500 [==============================] - 1s 1ms/sample - loss: 0.9281 - accuracy: 0.6760 - val_loss: 0.9393 - val_accuracy: 0.6903
Epoch 3/10
500/500 [==============================] - 1s 1ms/sample - loss: 0.6707 - accuracy: 0.7620 - val_loss: 0.7281 - val_accuracy: 0.7308
Epoch 4/10
500/500 [==============================] - 1s 1ms/sample - loss: 0.6219 - accuracy: 0.7640 - val_loss: 0.7165 - val_accuracy: 0.7255
Epoch 5/10
500/500 [==============================] - 1s 1ms/sample - loss: 0.5453 - accuracy: 0.8200 - val_loss: 0.6393 - val_accuracy: 0.7704
Epoch 6/10
500/500 [==============================] - 1s 1ms/sample - loss: 0.5058 - accuracy: 0.8140 - val_loss: 0.6567 - val_accuracy: 0.7502
Epoch 7/10
500/500 [==============================] - 1s 1ms/sample - loss: 0.4578 - accu

3000/3000 [==============================] - 1s 474us/sample - loss: 0.9760 - accuracy: 0.6487 - val_loss: 0.6682 - val_accuracy: 0.7457
Epoch 2/10
3000/3000 [==============================] - 1s 251us/sample - loss: 0.5749 - accuracy: 0.7873 - val_loss: 0.4903 - val_accuracy: 0.8290
Epoch 3/10
3000/3000 [==============================] - 1s 253us/sample - loss: 0.4553 - accuracy: 0.8343 - val_loss: 0.5057 - val_accuracy: 0.8159
Epoch 4/10
3000/3000 [==============================] - 1s 240us/sample - loss: 0.4242 - accuracy: 0.8440 - val_loss: 0.4248 - val_accuracy: 0.8460
Epoch 5/10
3000/3000 [==============================] - 1s 246us/sample - loss: 0.3571 - accuracy: 0.8730 - val_loss: 0.4285 - val_accuracy: 0.8499
Epoch 6/10
3000/3000 [==============================] - 1s 272us/sample - loss: 0.3158 - accuracy: 0.8840 - val_loss: 0.4273 - val_accuracy: 0.8469
Epoch 7/10
3000/3000 [==============================] - 1s 262us/sample - loss: 0.2752 - accuracy: 0.9033 - val_loss: 0.462

5500/5500 [==============================] - 1s 219us/sample - loss: 0.7946 - accuracy: 0.7142 - val_loss: 0.5383 - val_accuracy: 0.8009
Epoch 2/10
5500/5500 [==============================] - 1s 141us/sample - loss: 0.4874 - accuracy: 0.8245 - val_loss: 0.4548 - val_accuracy: 0.8338
Epoch 3/10
5500/5500 [==============================] - 1s 140us/sample - loss: 0.4079 - accuracy: 0.8542 - val_loss: 0.4643 - val_accuracy: 0.8233
Epoch 4/10
5500/5500 [==============================] - 1s 141us/sample - loss: 0.3415 - accuracy: 0.8740 - val_loss: 0.4109 - val_accuracy: 0.8438
Epoch 5/10
5500/5500 [==============================] - 1s 140us/sample - loss: 0.3030 - accuracy: 0.8915 - val_loss: 0.4137 - val_accuracy: 0.8462
Epoch 6/10
5500/5500 [==============================] - 1s 153us/sample - loss: 0.2675 - accuracy: 0.9065 - val_loss: 0.3928 - val_accuracy: 0.8616
Epoch 7/10
5500/5500 [==============================] - 1s 150us/sample - loss: 0.2291 - accuracy: 0.9162 - val_loss: 0.387

8000/8000 [==============================] - 1s 174us/sample - loss: 0.7373 - accuracy: 0.7294 - val_loss: 0.5732 - val_accuracy: 0.7765
Epoch 2/10
8000/8000 [==============================] - 1s 98us/sample - loss: 0.4481 - accuracy: 0.8345 - val_loss: 0.5121 - val_accuracy: 0.8205
Epoch 3/10
8000/8000 [==============================] - 1s 93us/sample - loss: 0.3706 - accuracy: 0.8677 - val_loss: 0.3741 - val_accuracy: 0.8640
Epoch 4/10
8000/8000 [==============================] - 1s 115us/sample - loss: 0.3197 - accuracy: 0.8815 - val_loss: 0.3512 - val_accuracy: 0.8765
Epoch 5/10
8000/8000 [==============================] - 1s 95us/sample - loss: 0.2705 - accuracy: 0.9006 - val_loss: 0.3806 - val_accuracy: 0.8625
Epoch 6/10
8000/8000 [==============================] - 1s 97us/sample - loss: 0.2392 - accuracy: 0.9128 - val_loss: 0.3731 - val_accuracy: 0.8720
Epoch 7/10
8000/8000 [==============================] - 1s 94us/sample - loss: 0.2132 - accuracy: 0.9236 - val_loss: 0.3537 - v

In [34]:
plt.style.use('ggplot')
ax = sns.scatterplot(x='ratio', y='accuracy', data=results_3a)

AttributeError: module 'seaborn' has no attribute 'style'

In [17]:
model = Sequential([
    Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)

print({'accuracy': accuracy, 'loss': loss})

Train on 60000 samples, validate on 10000 samples
Epoch 1/30
25952/60000 [===========>..................] - ETA: 3s - loss: 0.4932 - accuracy: 0.8219

KeyboardInterrupt: 